In [ ]:
!git clone https://github.com/Aanimesh003/resumeparser.git

Cloning into 'resumeparser'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 34 (delta 3), reused 33 (delta 2), pack-reused 0
Receiving objects: 100% (34/34), 6.19 MiB | 18.05 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
!pip install spacy_transformers
!pip install -U spacy_transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.4 MB/s eta 0:00:00


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
cv_data = json.load(open('/content/resumeparser/data/training/train_data_new.json','r'))

In [ ]:
!python -m spacy init fill-config /content/resumeparser/data/training/base_config.cfg /content/resumeparser/data/training/config.cfg

2023-08-12 08:04:24.678661: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/resumeparser/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# This function creates a DocBin object for training a spaCy model using text and annotation pairs. 
# It adds named entities to the Doc based on the annotations and writes any errors to a file.
# The DocBin object is returned for use in training a spaCy model.
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indicies = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indicies:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indicies += list(range(start, end))

      try:
        span = doc.char_span(start, end, label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str() + "  " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.set_ents(ents)
      db.add(doc)
    except:
      pass

  return db




In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(cv_data, test_size=0.3)

In [ ]:
# This code creates DocBin objects for training and test data using the `get_spacy_doc` function, saves them to files named 'train_data.spacy' and 'test_data.spacy', respectively, and writes any errors encountered during the creation of named entity spans to a file named 'error.txt'.
#  The file is opened with write mode and utf-8 encoding, and is closed after the DocBin objects are saved to files.
file = open('error.txt', 'w', encoding='utf-8')  # Specifying the  encoding as utf-8
db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()


100%|██████████| 365/365 [00:04<00:00, 88.28it/s] 


In [ ]:
!python -m spacy train /content/resumeparser/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-08-12 08:06:38.007420: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model che

In [ ]:
import spacy

nlp = spacy.load('/content/output/model-best')

In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.4 MB/s eta 0:00:00


In [ ]:
import fitz

filename = '/content/resumeparser/data/test/Alice Clark CV.pdf'
doc = fitz.open(filename)

text = ""
for page in doc:
    text += page.get_text()

text = text.strip()

In [ ]:
# output with 86% accuracy
import spacy

# Loading the  trained NER model
nlp = spacy.load('/content/output/model-last')

# Processing the text with the NER model
doc = nlp(text)

# Accessing  the named entities in the document
for ent in doc.ents:
    print(ent.text," ->>>>>  ", ent.label_)

Michael Smith  ->>>>>   NAME
Big Data/  ->>>>>   SKILLS
Manchester, UK-  ->>>>>   LOCATION
Microsoft - Manchester, UK.  ->>>>>   LOCATION
Web App  ->>>>>   SKILLS
The University of Manchester - UK  ->>>>>   UNIVERSITY


In [ ]:
# output with 83% accuracy with new dataset
import spacy

# Loading the  trained NER model
nlp = spacy.load('/content/output/model-last')

# Processing the text with the NER model
doc = nlp(text)

# Accessing  the named entities in the document
for ent in doc.ents:
    print(ent.text," ->>>>>  ", ent.label_)

Alice Clark  ->>>>>   Name
AI /  ->>>>>   SKILLS
Machine Learning  ->>>>>   SKILLS
Delhi  ->>>>>   Location
Software Engineer 
Microsoft  ->>>>>   WORKED AS
Microsoft  ->>>>>   Companies worked at
Microsoft  ->>>>>   Companies worked at
Microsoft  ->>>>>   Companies worked at
Microsoft  ->>>>>   Companies worked at
Microsoft  ->>>>>   Companies worked at
Microsoft  ->>>>>   Companies worked at
Indian Institute of Technology – Mumbai  ->>>>>   COLLEGE NAME


In [ ]:
#output with 64% accuracy
import spacy

# Loading the  trained NER model
nlp = spacy.load('/content/output/model-best')

# Processing the text with the NER model
doc = nlp(text)

# Accessing  the named entities in the document
for ent in doc.ents:
    print(ent.text," ->>>>>  ", ent.label_)


Alice Clark  ->>>>>   Name
Delhi  ->>>>>   Location
Microsoft  ->>>>>   Companies worked at
Software Engineer  ->>>>>   Designation
Microsoft  ->>>>>   Companies worked at
Microsoft  ->>>>>   Companies worked at
Microsoft  ->>>>>   Companies worked at
Microsoft  ->>>>>   Companies worked at
Microsoft  ->>>>>   Companies worked at
Microsoft  ->>>>>   Companies worked at
Microsoft  ->>>>>   Companies worked at
Indian Institute of Technology – Mumbai  ->>>>>   College Name
2001  ->>>>>   Graduation Year
Machine Learning, Natural Language Processing, and Big Data Handling 
 
ADDITIONAL INFORMATION 
Professional Skills 
• Excellent analytical, problem solving, communication, knowledge transfer and interpersonal 
skills with ability to interact with individuals at all the levels 
• Quick learner and maintains cordial relationship with project manager and team members and 
good performer both in team and independent job environments 
• Positive attitude towards superiors &amp; peers 
• Superv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

def folder_to_zip(folder_path, zip_file_path):
    # Create a new zip file
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Iterate through all files and subdirectories in the folder
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Add each file to the zip file
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Usage example
folder_path = '/content/output'  # Replace with your folder path in Colab
zip_file_path = '/content/new_model.zip'  # Replace with desired zip file path in Colab
folder_to_zip(folder_path, zip_file_path);




In [ ]:
!mv /content/new_model.zip /content/drive/MyDrive/nermodel
